# Introduction to Unit Metrics

In this notebook, we introduce the concept of *Unit Metric* and provide a step-by-step guide on how to define, execute and extract results from these measures. As an example, we use data from a customer churn use case to fit a binary classification model. To illustrate the application of these measures, we show how to run sklearn classification metrics as unit metrics, demonstrating their utility in quantifying model performance and risk.

In Model Risk Management (MRM), the primary objective is to identify, assess, and mitigate the risks associated with the development, implementation, and ongoing use of quantitative models. The process of measuring risk involves the understanding and assessment of evidence generated throw multiple tests across all the model development lifecycle stages, from data collection and data quality to model performance and explainability. 

### Evidence vs Risk

The distinction between evidence and quantifiable risk measures is a critical aspect of MRM. Evidence, in this context, refers to the outputs from various tests conducted throughout the model lifecycle. For instance, a table displaying the number of missing values per feature in a dataset is a form of evidence. It shows where data might be incomplete, which can affect the model's performance and reliability. Similarly, a Receiver Operating Characteristic (ROC) curve is a graphical plot that illustrates the diagnostic ability of a binary classifier system as its discrimination threshold is varied. The curve is evidence of the model's classification performance.

However, these pieces of evidence do not offer a direct measure of risk. To quantify risk, one must derive metrics from this evidence that reflect the potential impact on the model's performance and the decisions it informs. For example, the missing data rate, calculated as the percentage of missing values in the dataset, is a quantifiable risk measure that indicates the risk associated with data quality. Similarly, the accuracy score, which measures the proportion of correctly classified labels, acts as an indicator of performance risk in a classification model.

### Unit Metric

A *Unit Metric* is a single value measure that is used to identify and monitor risks arising from the development of Machine Learning or AI models. This metric simplifies evidence into a single actionable number, that can be monitored and compared over time or across different models or datasets. 

**Properties**
- They are the fundamental computation unit that returns a single value.
- They quantify risk and can be used to monitor and assess risks associated with a model's entire lifecycle.
- Measurable, relevant, and linked to risk areas and critical business processes - e.g., regulatory requirements, risk appetite, model performance, data quality.
- Standalone in nature, meaning they do not rely on other metrics for their calculation or interpretation.

Incorporating Unit Metrics into your ML workflow streamlines risk assessment, turning complex analyses into clear, actionable insights.

## Notebook Setup

In [1]:
import xgboost as xgb

%matplotlib inline

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place. 

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
import validmind as vm

vm.init(
  api_host = "https://api.dev.vm.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

2024-03-19 09:41:41,556 - INFO(validmind.api_client): Connected to ValidMind. Project: [Demo] Customer Churn Model - Initial Validation (clo0f697t003dn8rycwynwlox)


## Load the demo dataset

In this example, we load a demo dataset to fit a customer churn model. 

In [3]:
from validmind.datasets.classification import customer_churn as demo_dataset

print(f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}")

raw_df = demo_dataset.load_data()
raw_df.head()

Loaded demo dataset with: 

	• Target column: 'Exited' 
	• Class labels: {'0': 'Did not exit', '1': 'Exited'}


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Train a model for testing

We train a simple customer churn model for our test.

In [4]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [5]:
feature_columns = [col for col in test_df.columns if col != demo_dataset.target_column]
feature_columns

['CreditScore',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Geography_France',
 'Geography_Germany',
 'Geography_Spain']

## Compute Predictions

After the model is fitted, we compute model predictions and predictive probabilities, then add them to the customer churn dataset:

In [6]:
# Compute predictive probabilities for the test dataset
# Here, we only use the probabilities for the positive class (class 1)
predictive_probabilities = model.predict_proba(test_df.drop(demo_dataset.target_column, axis=1))[:, 1]

# Add the predictive probabilities as a new column to the test dataframe
test_df['PredictiveProbabilities'] = predictive_probabilities

# Add the predictions from the predictive probabilities as a new column to the test dataframe
test_df['Predictions'] = (predictive_probabilities > 0.5).astype(int)

# Display the first few rows of the updated dataframe to verify
test_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,PredictiveProbabilities,Predictions
3347,668,0,46,7,161806.09,1,1,1,173052.19,0,0,1,0,0.384697,0
2786,686,0,41,7,152105.57,2,0,1,132374.41,0,1,0,0,0.105136,0
7704,608,0,40,7,96202.32,1,0,0,161154.85,0,0,1,0,0.286090,0
857,420,0,34,1,135549.90,1,0,0,149471.13,1,0,1,0,0.404246,0
527,624,0,48,3,122388.38,2,0,0,30020.09,0,0,1,0,0.444074,0


## Initialize ValidMind objects

Once the datasets and model are prepared for validation, we initialize ValidMind `dataset` and `model`, specifying features and targets columns. The property `input_id` allows users to uniquely identify each dataset and model. This allows for the creation of multiple versions of datasets and models, enabling us to compute metrics by specifying which versions we want to use as inputs.

In [7]:
import validmind as vm

vm_test_ds = vm.init_dataset(
    input_id='test_dataset',
    dataset=test_df, 
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns, 
)

vm_model = vm.init_model(
    model=model,
    input_id="my_model"
)

2024-03-19 09:41:41,774 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


## Assign Predictions

**Assigning Pre-computed Predictions**

We use `vm_test` to incorporate a column named 'Predictions', which consists of pre-computed predictions associated with `vm_model`. The `assign_predictions` method facilitates the addition of multiple prediction columns as necessary. By linking these precomputed predictions to a specific model through this method, we establish a clear reference system, allowing for precise identification of the predictions needed for various computational tasks.

In [8]:
vm_test_ds.assign_predictions(
    model=vm_model, 
    prediction_column='Predictions'
)

In [9]:
vm_test_ds._extra_columns

{'prediction_columns': {'my_model': 'Predictions'}, 'group_by_column': None}

## Running Unit Metrics

### Computing F1 Score 

The following snippet shows how to set up and execute a unit metric implementation of the F1 score from `sklearn`. In this example, our objective is to compute F1 for the test dataset. Therefore, we specify `vm_test_ds` as the dataset in the inputs along with the `metric_id`. 

**Dataset to Metric Input Mapping**

To accurately compute the F1 score, it's essential to ensure that these columns are correctly aligned and contain the relevant data. The F1 score requires two inputs: 

- the predictions `y_pred` and 
- the true labels `y_true`
 

Since `vm_test_ds` has the capability to include multiple prediction columns, each linked to a different model. Therefore, it's essential to specify both the dataset for extracting the target column and the correct prediction column, as well as the model to ensure the selection of the appropriate prediction column for that specific model, referred to as `vm_model`.

When calculating the F1 score, it's essential to use the correct prediction column associated with `vm_model` within `vm_test_ds`. This prediction column is dynamically identified based on the model id, specified in `input_id`. 

In [10]:
metric_id = "validmind.unit_metrics.sklearn.classification.F1"

inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

result = vm.unit_metrics.run_metric(
    metric_id=metric_id,
    inputs=inputs,
)

### Accessing Metric Results

Once the metric computation is complete, the result object provides two key attributes: 

In [11]:
result.value

0.5788497217068647

In [12]:
result.summary

{'f1': 0.5788497217068647}

### Passing Parameters

When using the unit metric implementation of the F1 score from `sklearn`, it's important to note that this implementation supports all parameters of the original `sklearn.metrics.f1_score` function. This flexibility allows you to tailor the metric computation to your specific needs and scenarios. 

Below, we provide a brief description the parameters you can pass to customize the F1 score calculation:

- `average`: Specifies the averaging method for the F1 score. Common options include 'micro', 'macro', 'samples', 'weighted', or None. 
- `sample_weight`: Allows for weighting of samples. By default, it is None, but it can be an array of weights that are applied to the samples, useful for cases where some classes are more important than others.
- `zero_division`: Defines the behavior when there is a division by zero during F1 calculation. Options are 'warn', 'raise', or a numeric value like 0 or 1, indicating what value to set when encountering division by zero.

In [13]:
inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

params = {
    "average": "micro",
    "sample_weight": None,
    "zero_division": "warn"
}

result = vm.unit_metrics.run_metric(
    metric_id=metric_id, 
    inputs=inputs,
    params=params
)
result.value

0.858125

### Loading the Last Computed Value

Unit metrics are designed to optimize performance and efficiency by caching results of metric computations. When you execute a metric with the same signature —a unique combination of the metric ID, model, inputs, and parameters- a second time, validmind retrieves the result from its last computed value instead of recalculating it. This feature ensures faster access to metrics you've previously run and conserves computational resources.

**First Computation of Precision Metric**

In this first example, the precision metric is computed for the first time with a specific dataset. The result of this computation is stored in the cache.

In [14]:
metric_id = "validmind.unit_metrics.sklearn.classification.Precision"

inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

result = vm.unit_metrics.run_metric(
    metric_id=metric_id, 
    inputs=inputs, 
)
result.value

0.6995515695067265

In [15]:
result.summary

{'precision': 0.6995515695067265}

**Second Computation with the Same Signature**

In this second example, the same precision metric computation is requested again with the identical inputs. Since the signature (metric ID and inputs) matches the previous run, validmind loads the result directly from the cache instead of recomputing it.

In [16]:
result = vm.unit_metrics.run_metric(
    metric_id=metric_id, 
    inputs=inputs,  
)
result.value

0.6995515695067265

**Computation with a Changed Signature**

In this example, the signature is modified by adding parameters. This change prompts validmind to compute the metric anew, as the new signature does not match any stored result. The outcome is then cached, ready for any future requests with the same signature.

In [17]:
inputs = {
    "dataset": vm_test_ds,
    "model": vm_model
}

params = {
    "average": "micro",
    "sample_weight": None,
    "zero_division": "warn"
}

result = vm.unit_metrics.run_metric(
    metric_id=metric_id, 
    inputs=inputs,
)
result.value

0.6995515695067265

### Unit Metrics for Model Performance

In [18]:
metric_id = "validmind.unit_metrics.sklearn.classification.Accuracy"

inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

result = vm.unit_metrics.run_metric(
    metric_id=metric_id, 
    inputs=inputs,   
)
result.value


0.858125

In [19]:
result.summary

{'accuracy': 0.858125}

In [20]:
metric_id = "validmind.unit_metrics.sklearn.classification.Recall"

inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

result = vm.unit_metrics.run_metric(
    metric_id=metric_id, 
    inputs=inputs,
)
result.value

0.4936708860759494

In [21]:
result.summary

{'recall': 0.4936708860759494}

In [22]:
metric_id = "validmind.unit_metrics.sklearn.classification.ROC_AUC"

inputs = {
    "model": vm_model,
    "dataset": vm_test_ds
}

result = vm.unit_metrics.run_metric(
    metric_id=metric_id, 
    inputs=inputs,
)
result.value

0.7207451003588469

In [23]:
result.summary

{'roc_auc': 0.7207451003588469}

# Composing Complex Metrics from Individual Unit Metrics

#### Run Multiple Unit Metrics

In [24]:
from validmind.unit_metrics.composite import run_metrics

result = run_metrics(
    name="ModelPerformance",
    metric_ids=[
        "validmind.unit_metrics.sklearn.classification.F1",
        "validmind.unit_metrics.sklearn.classification.Precision",
        "validmind.unit_metrics.sklearn.classification.Recall",
        "validmind.unit_metrics.sklearn.classification.Accuracy",
        "validmind.unit_metrics.sklearn.classification.ROC_AUC"
    ],
    inputs=inputs,
)

In [25]:
result = run_metrics(
    name="ModelPerformance",
    metric_ids=[
        "validmind.unit_metrics.sklearn.classification.F1",
        "validmind.unit_metrics.sklearn.classification.Precision",
        "validmind.unit_metrics.sklearn.classification.Recall",
        "validmind.unit_metrics.sklearn.classification.Accuracy",
        "validmind.unit_metrics.sklearn.classification.ROC_AUC"
    ],
    inputs=inputs,
    output_template="""
    <h1>Model Performance</h1>
    <table>
        <thead>
            <tr>
                <th>Accuracy</th>
                <th>Precision</th>
                <th>Recall</th>
                <th>F1 Score</th>
                <th>ROC AUC</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>{{ value["accuracy"] | number }}</td>
                <td>{{ value["precision"] | number }}</td>
                <td>{{ value["recall"] | number }}</td>
                <td>{{ value["f1"] | number }}</td>
                <td>{{ value["roc_auc"] | number }}</td>
            </tr>
        </tbody>
    </table>
    """
)

In [26]:
result.result_id

'validmind.composite_metric.ModelPerformance'

In [27]:
# await result.log() # need to implement some way to log the result:
# for now we can log as its own metric result but we need to add support for unit metrics on the backend
# in which case, would we want to log the individual unit metrics and then save a normal metric result separately?
# also how would this get added to the template?
# feels like we really need a way to edit the template from the developer framework (at least be able to add metrics and output templates)

In [28]:
result.log()

In [34]:
await result.log_async()

<_GatheringFuture pending>

2024-03-19 09:42:32,508 - ERROR(validmind.api_client): Error logging metrics to ValidMind API
2024-03-19 09:42:32,509 - ERROR(validmind.api_client): Error logging metadata to ValidMind API
2024-03-19 09:42:32,510 - ERROR(validmind.api_client): Error logging metadata to ValidMind API


#### Template-Based Composite Metric

This is how the composite metric can be defined in the template:

In [30]:
vm.preview_template()

2024-03-19 09:41:44,074 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierInSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierInSamplePerformance'


Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

```yaml
        - content_type: composite_metric
          content_id: composite_metric.ModelPerformance
          unit_metrics:
            - validmind.unit_metrics.sklearn.classification.F1
            - validmind.unit_metrics.sklearn.classification.Precision
            - validmind.unit_metrics.sklearn.classification.Recall
            - validmind.unit_metrics.sklearn.classification.Accuracy
            - validmind.unit_metrics.sklearn.classification.ROC_AUC
          output_template: |
            <table>
                <thead>
                    <tr>
                        <th>Accuracy</th>
                        <th>Precision</th>
                        <th>Recall</th>
                        <th>F1 Score</th>
                        <th>ROC AUC</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>{{ value["accuracy"] }}</td>
                        <td>{{ value["precision"] }}</td>
                        <td>{{ value["recall"] }}</td>
                        <td>{{ value["f1"] }}</td>
                        <td>{{ value["roc_auc"] }}</td>
                    </tr>
                </tbody>
            </table>
```

In [31]:
vm.run_documentation_tests(
    inputs=inputs,
    section="model_evaluation",
    fail_fast=True,
)

2024-03-19 09:41:44,444 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierInSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierInSamplePerformance'


2024-03-19 09:41:44,656 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'training_test_degradation': (MissingRequiredTestInputError) datasets 'datasets' is a required input and must be passed as part of the test inputs dictionary.


BinaryClassificationTestSuite(sections=[TestSuiteSection(tests=[], section_id='__top_level__', description=None), TestSuiteSection(tests=[<validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2c44dfc10>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2c44df010>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2a60ca1d0>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2a5ed3f40>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2a5ed3fd0>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2a5ed3fa0>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2a5ed3f10>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2c105a050>, <validmind.vm_models.test_suite.test.TestSuiteTest object at 0x2c1059b40>], section_id='model_evaluation', description='Model Evaluation')])

#### Class-Based Composite Metric

In [32]:
from validmind.unit_metrics.composite import metric

@metric
class ModelPerformance:
    unit_metrics = [
        "validmind.unit_metrics.sklearn.classification.Accuracy",
        "validmind.unit_metrics.sklearn.classification.F1",
        "validmind.unit_metrics.sklearn.classification.Precision",
        "validmind.unit_metrics.sklearn.classification.Recall",
        "validmind.unit_metrics.sklearn.classification.ROC_AUC"
    ]

    def output(self, results):
        return f"""
        <h1>Model Performance</h1>
        <table>
            <tr>
                <th>Metric</th>
                <th>Value</th>
            </tr>
            <tr>
                <td>Accuracy</td>
                <td>{results["accuracy"]}</td>
            </tr>
            <tr>
                <td>F1</td>
                <td>{results["f1"]}</td>
            </tr>
            <tr>
                <td>Precision</td>
                <td>{results["precision"]}</td>
            </tr>
            <tr>
                <td>Recall</td>
                <td>{results["recall"]}</td>
            </tr>
            <tr>
                <td>ROC AUC</td>
                <td>{results["roc_auc"]}</td>
            </tr>
        </table>
        """

In [33]:
model_performance = ModelPerformance()
model_performance.run(inputs=inputs)

ValueError: metric_ids must be provided

In [ ]:
model_performance.output(model_performance.results)

AttributeError: 'ModelPerformance' object has no attribute 'results'

#### Alternative Composite Metric Class (w/ inheritance)

In [ ]:
from validmind.unit_metrics.composite import CompositeMetric

model_performance = CompositeMetric(
    unit_metrics = [
        "validmind.unit_metrics.sklearn.classification.Accuracy",
        "validmind.unit_metrics.sklearn.classification.F1",
        "validmind.unit_metrics.sklearn.classification.Precision",
        "validmind.unit_metrics.sklearn.classification.Recall",
        "validmind.unit_metrics.sklearn.classification.ROC_AUC"
    ],
    test_id="composite_metric.ModelPerformance",
    inputs=inputs,
)

In [ ]:
from pandas import DataFrame
from validmind.vm_models import Metric

__custom_test_registry = {
    "composite_metric.MyCompositeMetric": MyCompositeMetric,
}

class MyCompositeMetric(Metric):
    unit_metrics = [
        "validmind.unit_metrics.sklearn.classification.Accuracy",
        "validmind.unit_metrics.sklearn.classification.F1",
        "validmind.unit_metrics.sklearn.classification.Precision",
        "validmind.unit_metrics.sklearn.classification.Recall",
        "validmind.unit_metrics.sklearn.classification.ROC_AUC"
    ]

    def description(self):
        return "A composite metric that calculates multiple classification metrics for a model."

    def summary(self, metric_value: dict | list | DataFrame | None = None):
        return super().summary(metric_value)

```yaml
- content_type: metric
  content_id: composite_metric.MyCompositeMetric
```

In [ ]:
model_performance.run()

TypeError: Expected 'inputs' to be a dictionary.